<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/08_nlp_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-10-24 04:02:41--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-10-24 04:02:42 (33.3 MB/s) - ‘helper_functions.py’ saved [6733/6733]



In [2]:
from helper_functions import unzip_data,plot_loss_curves,compare_historys,create_tensorboard_callback

In [3]:
# Get the Dataset from kaggle NLP with Disaster Tweets
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-10-24 04:02:50--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.207, 108.177.121.207, 142.251.183.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.02s   

2025-10-24 04:02:51 (37.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

In [5]:
# Become One with Dataset (Visualize)
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
# Shuffle Dataset
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does Test Dataset Look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each classes
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [11]:
# How many Total Samples
len(train_df),len(test_df)

(7613, 3263)

# **Visualize some random training examples**

In [12]:
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
    _,text,target=row
    print(f'Target : {target} \n ',"(real disaster)" if target>0 else "not real disaster")
    print(f'Text : \n {text}')
    print('---\n')

Target : 1 
  (real disaster)
Text : 
 A brief violent storm swept through the Chicago area Sunday afternoon leading to one death and an evacuation of Lollapalooza and more
---

Target : 0 
  not real disaster
Text : 
 Owner of Chicago-Area Gay Bar Admits to Arson Scheme: Frank Elliott pleaded guilty to hiring an arsonist to to... http://t.co/jCFEhrHLq8
---

Target : 1 
  (real disaster)
Text : 
 Senators calling for emergency housing: Boxer Feinstein back plan to move #homeless vets to VA campus http://t.co/Gm80X3vutf
---

Target : 0 
  not real disaster
Text : 
 @DmoneyDemi I had my meltdown yesterday.  I'm going to miss you so much. You are forsure my DTB for life. When I get back watchout ??
---

Target : 1 
  (real disaster)
Text : 
 .@Vagersedolla visits villages recently bombed by Turkey and finds people fed up with the PKK http://t.co/UUWEiKD7sP
---



In [13]:
#Splitting data into Train and Validation
from sklearn.model_selection import train_test_split

train_sentances,val_sentances,train_labels,val_labels=train_test_split(
                                                      train_df_shuffled['text'].to_numpy(),
                                                      train_df_shuffled['target'].to_numpy(),
                                                      test_size=0.1,
                                                      random_state=42)

In [14]:
# Check lengths
len(train_sentances),len(val_sentances),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

# **Convert Text into Numbers**

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer=TextVectorization(
                                  max_tokens=10000, # Set max_tokens to a value
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  pad_to_max_tokens=True,
                                   )

In [16]:
max_vocab_length=100   # Max number of words to have in vocabulary
max_length=15  # Max length our sequence will be
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=max_vocab_length,
                                  pad_to_max_tokens=True)

In [17]:
# Fit the Text Vectorizer
text_vectorizer.adapt(train_sentances)

In [18]:
# Create a sample sentance and tokenize it
sample_sentance=random.choice(train_sentances)
print(f' The Original Sentance \n {sample_sentance} \n The Text Vectorized Version \n')
text_vectorizer([sample_sentance])

 The Original Sentance 
 (OFFICIAL VID) &gt; #DoubleCups &gt;&gt; https://t.co/lfKMTZaEkk &gt;&gt; @TrubGME Prod @THISIZBWRIGHT &gt;&gt; #ARMAGEDDON                 . 
 The Text Vectorized Version 



<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [19]:
# Get uniqe words in vocab
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f'The Number of words in vocab are : {len(words_in_vocab)}')
print(f'The Five most common words in vocab are : {top_5_words}')
print(f'The Five least words in vocab are : {bottom_5_words}')

The Number of words in vocab are : 100
The Five most common words in vocab are : ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
The Five least words in vocab are : [np.str_('buildings'), np.str_('rt'), np.str_('first'), np.str_('cant'), np.str_('see')]


In [20]:
# Creating a embedding layer that turns text vectorized into the embeding vectors
from tensorflow.keras.layers import Embedding

embedding=Embedding(input_dim=max_vocab_length,
                    output_dim=128,
                    )

In [21]:
# Get Random Sentance from trainning set
random_sentance=random.choice(train_sentances)
print(f'Original Text \n {random_sentance}\n')
sample_embedding=embedding(text_vectorizer([random_sentance]))
sample_embedding

Original Text 
 Look: #I have collapsed #after attempting to munch an endangered species.



<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
array([[[ 4.6506491e-02,  1.4164988e-02, -1.0480881e-03, ...,
          3.3488993e-02, -1.6133558e-02, -1.3996802e-02],
        [ 4.6285477e-02,  5.7812557e-03, -3.2888256e-02, ...,
         -5.6624413e-06,  3.8006220e-02, -3.7045132e-02],
        [ 9.0920329e-03, -4.0367387e-02,  7.9524145e-03, ...,
          3.5651494e-02, -4.9711801e-02,  2.9194761e-02],
        ...,
        [ 4.1544262e-02,  3.8823750e-02, -4.3369643e-03, ...,
          3.6131728e-02, -4.8943687e-02, -9.9865906e-03],
        [ 4.1544262e-02,  3.8823750e-02, -4.3369643e-03, ...,
          3.6131728e-02, -4.8943687e-02, -9.9865906e-03],
        [ 4.1544262e-02,  3.8823750e-02, -4.3369643e-03, ...,
          3.6131728e-02, -4.8943687e-02, -9.9865906e-03]]], dtype=float32)>

# ***Lets Now Create our Baseline Model***
its common to use ML model first or non DL model and then switch to DL model

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([
    ('tfid',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

model_0.fit(train_sentances,train_labels)
baseline_score=model_0.score(val_sentances,val_labels)

print(f'Our Baseline Model Acheives Accuracy of : {baseline_score*100:.2f}')

Our Baseline Model Acheives Accuracy of : 79.27


In [23]:
# Model Predictions
baseline_preds=model_0.predict(val_sentances)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# **Creating an evaluation function**

In [24]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support


def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true=y_pred,y_pred=y_pred)
  model_precission,model_recall,model_f1,_=precision_recall_fscore_support(y_true=y_true,average='weights')

  model_results={
     "accuracy":model_accuracy,
     "precsion":model_precission,
     "recall":model_recall,
     'f1_score':model_f1
  }
  return model_results